## Movie Analysis

### Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('disney_movie_data.csv', index_col=False).drop(['Unnamed: 0'], axis=1)
df.columns
# df.head()

Index(['Title', 'Production company', 'Distributed by', 'Release date',
       'Running time', 'Country', 'Language', 'Box office',
       'Release date (datetime)', 'Directed by', 'Written by', 'Based on',
       'Produced by', 'Starring', 'Music by', 'Release dates', 'Budget',
       'Story by', 'Narrated by', 'Cinematography', 'Edited by', 'Languages',
       'Screenplay by', 'Countries', 'Color process', 'Production companies',
       'Japanese', 'Hepburn', 'Adaptation by', 'Animation by', 'Traditional',
       'Simplified', 'Original title', 'Layouts by', 'Original concept by'],
      dtype='object')